# Get sequence info (get_seq_splicesite.py)

The following file is the file that contains the function to incorporate sequence information for each splice junction

In [1]:
#!usr/local/bin
import pandas as pd
import numpy as np
from sys import argv
import subprocess

### Inputs and Intital setup of function

The input to this function is all the splice junctions given by the STAR file and the fasta file containing sequence info and is compatible with bedtools getfasta for alignment

Steps of function:
1. Take splice junctions from STAR file and add a strand column containing + or - string

In [2]:
def sequence_splice_site(sp_junc, fasta_file):
    part = sp_junc
    part = part[['chr','first_base','last_base','motif','STAR_annotation','strand']]
    part['first_base'] = part['first_base']
    part['strand'] = part['strand'].apply(lambda x: '+' if x ==1 else ('-' if x==2 else np.nan))

2. Separate the splice junctions into two dataframes.
    - One containing the splice site found at the start base
    - The other containing the splice site found at the end base
3. Combine the two dataframes into one and use bedtools getfasta to take the strand info, and cooridante for each splice site to get the necessary sequence information
    

In [3]:
    cat1 = pd.DataFrame(columns = ['chr','start','end','def','ss','strand'])
    cat2 = pd.DataFrame(columns = ['chr','start','end','def','ss','strand'])
    cat1['chr'] = part['chr']
    cat1['start'] = part['first_base'] - 26
    cat1['end'] = part['first_base'] +25
    cat1['strand'] = part['strand']
    cat1['ss'] = part['strand'].apply(lambda x: 3 if x =='-' else 5)
    cat1['def'] = part.index +',' + cat1['ss'].astype(str)

    cat2['chr'] = part['chr']
    cat2['start'] = part['last_base'] -26
    cat2['end'] = part['last_base'] +25
    cat2['strand'] = part['strand']
    cat2['ss'] = part['strand'].apply(lambda x: 5 if x =='-' else 3)
    cat2['def'] = part.index +',' + cat2['ss'].astype(str)

    fin = pd.concat([cat1,cat2])

    fin.to_csv('data/bedfileex2.bed', sep ='\t', header = False, index = False)
    bash_cmd = "bedtools getfasta -fi %s -bed data/bedfileex2.bed -s -name -tab" % fasta_file
    process= subprocess.Popen(bash_cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    results = output.decode("utf-8")

NameError: name 'part' is not defined

### Processing output from bedtools getfasta command

Read the result of the bedtools getfasta command and make a dataframe out of it. This dataframe will be what is outputted by the program. The dataframe will contain which splice site sequnce is associated with either the. 5' or 3' splice site for each splice junction

In [ ]:
    seqs = pd.DataFrame([x.split('\t') for x in results_in_list], columns = ["names", 'seq'])
    seqs['id_jnc'] = seqs['names'].str.split(':', n=1, expand = True)[0]
    seqs[['JNC_ID', 'site']] = seqs.id_jnc.apply(lambda x: pd.Series(str(x).split(',')))
    five_pr_seq = seqs.loc[seqs.site.isin(['5']), 'seq']
    three_pr_seq = seqs.loc[seqs.site.isin(['3']), 'seq']
    return seqs